In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from __future__ import annotations

In [18]:
from utils import load_data
from constants import *
import pandas as pd
import numpy as np

In [4]:
YEARS = [2014, 2015]
reddit = {}

In [5]:
for year in YEARS:
    reddit[year] = load_data(DATA_PATH, year=year, tokenize=False, comp="bz2", dev=True)

Loading data of 2014...
Loading data of 2015...


In [11]:
users = {}


for year in YEARS:
    print(year)
    for index, row in reddit[year].iterrows():
        if not row['subreddit'] in DEM_SUBREDDITS and not row['subreddit'] in REP_SUBREDDITS:
            continue
        else:
            try:
                users[row['author']].append(row['subreddit'])
            except KeyError:
                users[row['author']] = [row['subreddit']]
        

2014
2015


In [12]:
def rep_or_dem(subreddits_list):
    score = 0
    for subreddit in subreddits_list:
        if subreddit in DEM_SUBREDDITS:
            score += 1
        elif subreddit in REP_SUBREDDITS:
            score -= 1
    if score > 0: 
        return 'D'
    elif score < 0:
        return 'R'
    return 'N'

In [15]:
data = []
for year in YEARS:
    print('YEAR -> ', year)
    for index, row in reddit[year].iterrows():
        if row['subreddit'] == 'politics':
            try:
                affiliation = rep_or_dem(users[row['author']])
                new_line = [row['author'], row['body'], row['created_utc'], affiliation]
                data.append(new_line)
            except KeyError:
                continue
                
output_dataframe = pd.DataFrame(data, columns= ['author', 'post', 'time', 'politics'])
output_dataframe.to_csv('reduced_data.csv')

YEAR ->  2014
YEAR ->  2015


In [23]:
output_dataframe

,author,post,time,politics
0,justnfree,youre answering because its a good question to...,2014-01-02,R
1,NoMoreNicksLeft,but not whole . and even on the genetically di...,2014-01-04,R
2,liatris,"then look it up , quit expecting other people ...",2014-01-26,R
3,Melloz,he cant declassify it due to treaty obligation...,2014-01-31,R
4,IAmNotAPsychopath,logical fallacies . . . logical fallacies ever...,2014-01-01,R
...,...,...,...,...
123,cp5184,"and if a republican gets elected , say hello t...",2015-01-16,R
124,BasedGodCometh,youll never have a 3rd party candidate win wit...,2015-01-19,R
125,auryn0151,link america has done a great job implementing...,2015-01-05,R
126,humblues,i agree this is not a big deal and in a vacuum...,2015-01-01,R
